# Projeto 1 - Ciência dos Dados

Nome: Henrique Rocha Bomfim

Nome: Nicholas Morales Balkins

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [10]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

In [6]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

os.chdir("..\data")

Esperamos trabalhar no diretório
c:\Users\Henri\Documents\Insper\Ciência de Dados\Projeto 1\22-2a-cd-p1-grupo_henriquerb1\data


Carregando a base de dados com as notícias classificadas manualmente:

In [7]:
filename = 'dados.xlsx'

In [8]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Titulo,Descrição,Pagina,Target
0,Ucrânia oferece recompensa “generosa” para que...,A Ucrânia está de olho nas carteiras de cripto...,68,0
1,"Só 12,9% da oferta disponível de bitcoin no me...",Conforme o preço do bitcoin (BTC) continua sub...,100,0
2,Presidente do Paraguai veta lei que regula as ...,"O presidente do Paraguai, Mario Abdo Benítez, ...",4,0
3,Corretora de criptomoedas Coinbase vai entrar ...,"A Coinbase, a maior corretora de criptomoedas ...",5,0
4,Por que criptomoedas e esports estão cada vez ...,Quando a corretora de criptomoedas FTX anuncio...,13,0


In [9]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Titulo,Descrição,Pagina,Target
0,Binance libera compra de oito criptomoedas com...,A Binance liberou oito criptomoedas que agora ...,73,0
1,Veja qual criptomoeda pode “herdar” os minerad...,"Holders de ETC, a criptomoeda da rede Ethereum...",5,0
2,Venezuela vai punir quem negociar criptomoedas...,Os venezuelanos que negociam criptomoedas em e...,36,0
3,Corretora de criptomoedas desiste de patrocina...,A corretora Crypto.com desistiu de um enorme a...,3,0
4,Golpistas lançam falsa criptomoeda da empresa-...,Nos últimos dias começou a circular no mercado...,71,0


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação das notícias (Target).

Criptomoedas -> Se foi aderida para algum processo ou não, e se não se aplica à notícia.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [34]:
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    #text_subbed = re.sub('(\s+)(de|em|do|para)(\s+)','\1\3',text_subbed)
    return text_subbed

train_0 = train.loc[train["Target"] == 0,:]
train_1 = train.loc[train["Target"] == 1,:]

train_0_titulos = ''
for palavra in train_0.Titulo:
    train_0_titulos += palavra
train_0_titulos = cleanup(train_0_titulos).split()
train_0_titulos

train_1_titulos = ''
for palavra in train_1.Titulo:
    train_1_titulos += palavra
train_1_titulos = cleanup(train_1_titulos).split()

serie_train_0_titulos = pd.Series(train_0_titulos)
serie_train_1_titulos = pd.Series(train_1_titulos)


print(serie_train_0_titulos.value_counts(),"\n")
print(serie_train_1_titulos.value_counts())

de                    383
criptomoedas          246
em                    113
do                    111
para                  110
                     ... 
Lucas                   1
presidenteYouTuber      1
divulgação              1
preçoSecretária         1
esquentar               1
Length: 1846, dtype: int64 

de                  97
criptomoedas        61
e                   54
do                  33
em                  33
                    ..
entra                1
falência             1
beira                1
criptomoedasMais     1
depois               1
Length: 662, dtype: int64


In [37]:
tabela_absoluto_train_0 = serie_train_0_titulos.value_counts()
tabela_absoluto_train_1 = serie_train_1_titulos.value_counts()
tabela_relativa_train_0 = serie_train_0_titulos.value_counts(True)
tabela_relativa_train_1 = serie_train_1_titulos.value_counts(True)
print(tabela_relativa_train_0,"\n")
print(tabela_relativa_train_1)

de                    0.079116
criptomoedas          0.050816
em                    0.023342
do                    0.022929
para                  0.022723
                        ...   
Lucas                 0.000207
presidenteYouTuber    0.000207
divulgação            0.000207
preçoSecretária       0.000207
esquentar             0.000207
Length: 1846, dtype: float64 

de                  0.065101
criptomoedas        0.040940
e                   0.036242
do                  0.022148
em                  0.022148
                      ...   
entra               0.000671
falência            0.000671
beira               0.000671
criptomoedasMais    0.000671
depois              0.000671
Length: 662, dtype: float64


In [56]:
portugues = train_0_titulos + train_1_titulos
serie_portugues = pd.Series(portugues)
tabela_portugues_relativa = serie_portugues.value_counts
portugues_soma = tabela_absoluto_train_1.sum() + tabela_absoluto_train_0.sum()
prob0 = tabela_absoluto_train_0.sum()/portugues_soma
prob1 = tabela_absoluto_train_1.sum()/portugues_soma
print(prob0,"\n",prob1)
print(prob0 + prob1)

0.7646501342599905 
 0.2353498657400095
1.0


de                0.075817
criptomoedas      0.048492
e                 0.025746
em                0.023061
do                0.022745
                    ...   
autoriza          0.000158
agostoPolícia     0.000158
tráficoEmpresa    0.000158
suposto           0.000158
web3              0.000158
Length: 2198, dtype: float64

In [ ]:
probFraseDado0 = 1
probFraseDado1 = 1
prob_frase = tabela_portugues_relativa
frase_lista = frase.split()

for palavra in frase_lista:
    probFraseDado1 = probFraseDado1 * tabela_relativa_train_1[palavra]
    probFraseDado0 = probFraseDado0 * tabela_relativa_train_0[palavra]
print(probFraseDado1)
print(probFraseDado0)

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações das notícias entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas notícias. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas caterogias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por caterogia (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**